In [34]:
#Using Hashtags and Regex, tag tweets by candidate mention

#NOTE:  hashtags do not seem to add much so leaving them out for now, code commented out.

In [1]:
keywords = {}

keywords['Trump'] = ['trump']
keywords['Clinton'] = ['hillary', 'clinton']
keywords['Sanders'] = ['bern', 'sanders']
keywords['Cruz'] = ['cruz']
keywords['Rubio'] = ['marco', 'rubio']
keywords['Kasich'] = ['kasich']
keywords['Bush'] = ['jebbush','jeb', '#bush']
keywords['Christie'] = ['chrischristie', 'chris christie','christie2016','#christie']
keywords['Huckabee'] = ['huckabee']
keywords['Omally'] = ['omally']
keywords['Randpaul'] = ['rand paul','randpaul']
keywords['Fiorina'] = ['carly', 'carlley', 'fiorina']
keywords['Carson'] = ['ben carson', 'bencarson', 'bencarson', '#carson', 'bencarson2016', 'carson2016', 'votecarson', 'bencarson4prez', 'carson4prez', 'carsonstrong', 'respectcarson', 'drbencarson']


In [3]:
# #selected hashtags pre generated and stored in files ordered by frequency
# import pandas as pd
import pandas as pd

hts={}

for keyword in keywords:

    df2  = pd.read_csv('keywords/'+keyword+'.csv')
    
    hts[keyword] = list(df2['hashtag'].values[0:10])

    print hts[keyword]


['berniesanders', 'sanders', 'sanders2016', 'berniesanders2016', 'berniesandersforsc', 'berniesandersforpresident2016', 'sandersonhardball', 'janesanders', 'sanderstrumpdebatenow', 'berniesandersforpresident']
['bencarson', 'carson', 'bencarson2016', 'carson2016', 'votecarson', 'bencarson4prez', 'carson4prez', 'carsonstrong', 'respectcarson', 'drbencarson']
['hillaryclinton', 'clinton', 'billclinton', 'clintons', 'hilaryclinton', 'clintonemails', 'clinton2016', 'clintonfoundation', 'hillaryclintonlies', 'clintoncash']
['trump', 'trump2016', 'nevertrump', 'trumptrain', 'alwaystrump', 'donaldtrump', 'votetrump', 'dumptrump', 'realdonaldtrump', 'votetrump2016']
['randpaul', 'randpaul2016', 'randpaul2020', 'voterandpaul', 'randpaul2024', 'senrandpaul', 'imissrandpaul', 'belikerandpaul', 'randpaul16', 'randpaulcorvette']
['rubio', 'marcorubio', 'rubio2016', 'neverrubio', 'microsoftrubiofraud', 'robotrubio', 'rubiorobots', 'norubio', 'littlerubio', 'dumprubio']
['jebbush', 'bush', 'georgewbu

In [4]:
keywordstrings = {}

for key, value in keywords.iteritems():
    keywordstrings[key] = '|'.join(value)

keywordstrings

{'Bush': 'jebbush|jeb|#bush',
 'Carson': 'ben carson|bencarson|bencarson|#carson|bencarson2016|carson2016|votecarson|bencarson4prez|carson4prez|carsonstrong|respectcarson|drbencarson',
 'Christie': 'chrischristie|chris christie|christie2016|#christie',
 'Clinton': 'hillary|clinton',
 'Cruz': 'cruz',
 'Fiorina': 'carly|carlley|fiorina',
 'Huckabee': 'huckabee',
 'Kasich': 'kasich',
 'Omally': 'omally',
 'Randpaul': 'rand paul|randpaul',
 'Rubio': 'marco|rubio',
 'Sanders': 'bern|sanders',
 'Trump': 'trump'}

In [38]:
prequery = """CREATE CONSTRAINT ON (candidate:Candidate_{0}) ASSERT candidate.name IS UNIQUE"""

In [5]:
# query="""MATCH (t:Tweet_{0})<-[:TAGS]-(h:Hashtag_{0}) \
# WHERE ((h.name in {1}) OR (t.text =~ \"(?ism).*({2}).*\")) \
# SET t.mentions_{3} = true \
# RETURN count(t)"""

query="""MATCH (t:Tweet_{0}) \
WHERE (t.text =~ \"(?ism).*({1}).*\") \
SET t.mentions_{2} = true \
RETURN count(t)"""

In [6]:
partitions = ['2015_10_11', '2015_10_12', '2015_10_13', '2015_10_14', '2015_10_15',\
              '2016_1_26', '2016_1_27', '2016_1_28', '2016_1_29', '2016_1_30', '2016_2_2','2016_2_3', \
              '2016_2_4', '2016_2_5', '2016_2_6', '2016_2_7', '2016_2_8', '2016_2_9', '2016_2_10',\
              '2016_2_11', '2016_2_12', '2016_2_13', '2016_2_14', '2016_2_15', \
              '2016_2_23', '2016_2_24', '2016_2_25', '2016_2_26','2016_2_27', '2016_3_1', '2016_3_2',\
             '2016_3_3', '2016_3_4', '2016_3_5', '2016_3_6', '2016_3_7', '2016_3_8', '2016_3_9',\
             '2016_3_10', '2016_3_11']

# partitions = ['2016_2_24', '2016_2_25', '2016_2_26', '2016_3_1', '2016_3_2',\
#              '2016_3_3', '2016_3_4', '2016_3_5', '2016_3_6', '2016_3_7', '2016_3_8', '2016_3_9',\
#              '2016_3_10', '2016_3_11']



In [7]:
from neo4j.v1 import GraphDatabase, basic_auth
import time


def runprequery(driver, prequery):
    result = None
    try:
        session = driver.session()
        result = session.run(prequery)
        session.close()
        print 'completed'
    except:
        session.close()
        print 'Could not run query'


#driver = GraphDatabase.driver("bolt://sauce5.sdsc.edu", auth=basic_auth("neo4j", "lajolla"))
print 'opening driver...'
#driver = GraphDatabase.driver("bolt://sauce5.sdsc.edu", auth=basic_auth("neo4j", "lajolla"))
driver = GraphDatabase.driver("bolt://127.0.0.1")
print 'after opening driver...'

for partition in partitions:
#for partition in ['2016_3_2']:

#     prequery_formatted = prequery.format(partition)
#     print prequery_formatted
    
    #success = runprequery(driver,prequery_formatted)
        
    for candidate, keywords in keywordstrings.iteritems():
        
        start = time.time()
        
        #hashtags = hts[candidate]
        
        query_formatted = query.format(partition,keywords,candidate)
        print query_formatted
        
        session = driver.session()
        result = session.run(query_formatted)

        for record in result:
            print partition, candidate, record[0]

        session.close()
        
        end = time.time()
        print end - start

        

opening driver...
after opening driver...
MATCH (t:Tweet_2015_10_11) WHERE (t.text =~ "(?ism).*(huckabee).*") SET t.mentions_Huckabee = true RETURN count(t)
2015_10_11 Huckabee 92
0.396808862686
MATCH (t:Tweet_2015_10_11) WHERE (t.text =~ "(?ism).*(bern|sanders).*") SET t.mentions_Sanders = true RETURN count(t)
2015_10_11 Sanders 5756
0.418828964233
MATCH (t:Tweet_2015_10_11) WHERE (t.text =~ "(?ism).*(hillary|clinton).*") SET t.mentions_Clinton = true RETURN count(t)
2015_10_11 Clinton 10351
0.434485912323
MATCH (t:Tweet_2015_10_11) WHERE (t.text =~ "(?ism).*(trump).*") SET t.mentions_Trump = true RETURN count(t)
2015_10_11 Trump 17298
0.382841825485
MATCH (t:Tweet_2015_10_11) WHERE (t.text =~ "(?ism).*(cruz).*") SET t.mentions_Cruz = true RETURN count(t)
2015_10_11 Cruz 2014
0.261349916458
MATCH (t:Tweet_2015_10_11) WHERE (t.text =~ "(?ism).*(rand paul|randpaul).*") SET t.mentions_Randpaul = true RETURN count(t)
2015_10_11 Randpaul 256
0.343826055527
MATCH (t:Tweet_2015_10_11) WHERE 